In [2]:
import json
import urllib.request
import time
import datetime
today = datetime.date.today()
import tweepy as tp
from config import consumer_key,consumer_secret,access_token,access_secret

In [3]:
gameid = 2021020270
gamedetail = []
starturl = 'https://statsapi.web.nhl.com/api/v1/game/'
endurl = '/feed/live.json'
auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tp.API(auth)

In [4]:
# time.sleep(3600)
# Use for delay start
pullurl = starturl + str(gameid) + endurl
pullurl

'https://statsapi.web.nhl.com/api/v1/game/2021020270/feed/live.json'

In [ ]:
while gameid <= 2021020317:
    pullurl = starturl + str(gameid) + endurl
    with urllib.request.urlopen(pullurl) as url:
        obj = json.loads(url.read().decode())
    allPlays = obj["liveData"]["plays"]['allPlays']
    teamStats = obj["liveData"]["boxscore"]["teams"]
    game_date = obj["gameData"]["datetime"]["dateTime"]
    home_team = obj["gameData"]["teams"]["home"]["name"]
    away_team = obj["gameData"]["teams"]["away"]["name"]
    game_date = game_date.split("T")[0]
    tomorrow = today + datetime.timedelta(days = 1)
    print(f'{gameid}-{obj["gameData"]["status"]["abstractGameState"]}-{game_date}-{tomorrow} -- {home_team} vs {away_team}')
    # print(obj["gameData"]["status"]["abstractGameState"])
    x=1
    period = 1
    while obj["gameData"]["status"]["abstractGameState"] != "Final" and (home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings'):
        #print(x)
        #x=x+1
        pullurl = starturl + str(gameid) + endurl
        try:
            with urllib.request.urlopen(pullurl) as url:
                obj = json.loads(url.read().decode())
        except:
            print(f'The URL is broke, bruh. Try again.')
        allPlays = obj["liveData"]["plays"]['allPlays']
        hometeamscore = 0
        awayteamscore = 0
        for play in allPlays:
            period = play['about']['ordinalNum']
            event = play['result']['event']
            timeee = play['about']['periodTime']
            description = play['result']['description']
            if event == 'Goal':
                if str(home_team) == 'Los Angeles Kings':
                    if play['team']['name'] == 'Los Angeles Kings':
                        hometeamscore += 1
                    else:
                        awayteamscore += 1
                if str(home_team) != 'Los Angeles Kings':
                    if play['team']['name'] != 'Los Angeles Kings':
                        hometeamscore += 1
                    else:
                        awayteamscore += 1
            if f'Period: {period}-{timeee}, {event}' not in gamedetail:
                gamedetail.append(f'Period: {period}-{timeee}, {event}')
                if event == 'Period Ready' or event == 'Hit' or event == 'Giveaway' or event == 'Takeaway' or event == 'Game Scheduled':
                    print(f'NOT TWEETED---Period: {period}-{timeee}, {event}-{description}')
                    #Period Ready
                    #Giveaway
                    #Takeaway
                    #Hit
                #elif event == 'Game Scheduled':
                    #homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                    #awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                    #print(f'Scheduled Game: {home_team} at {away_team} #GoKingsGo')
                    #api.update_status(f'Scheduled Game: {home_team} versus {away_team} #GoKingsGo')
                    #time.sleep(8)
                elif event == 'Period Start':
                    try:
                        api.update_status(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        print(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET --- Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    time.sleep(8)
                    #Period Start
                elif event == 'Faceoff' or event == 'Blocked Shot' or event == 'Stoppage' or event == 'Period Official' or event == 'Game End' or event == 'Game Official' or event == 'Shot' or event == 'Penalty' or event == 'Missed Shot' :
                    try:
                        api.update_status(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                        print(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET -- Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                    time.sleep(8)
                    #Faceoff
                    #Blocked Shot
                    #Shot
                    #Penalty
                    #Missed Shot
                    #Stoppage
                    #Period Official
                    #Game End
                    #Game Official
                elif event == 'Goal':
                    try:
                        api.update_status(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        print(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET --- Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                    time.sleep(8)
                    #Goal
                elif event == 'Period End' and (period == '1st' or period == '2nd'):
                    try:
                        api.update_status(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        print(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET --- Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    time.sleep(1080)
                    #Period End
                elif event == 'Period End' and (period == '3rd'):
                    try:
                        api.update_status(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        print(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET --- End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    time.sleep(8)
                elif event == 'Period End' and (period == 'OT'):
                    try:
                        api.update_status(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        print(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET --- End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    time.sleep(8)
                else:
                    time.sleep(0)
        print(f'{home_team}-{hometeamscore}, {away_team}-{awayteamscore}')
    gameid += 1

2021020279-Live-2021-11-22-2021-11-22 -- Los Angeles Kings vs Arizona Coyotes
Period 2nd End --- Score: Los Angeles Kings-1, Arizona Coyotes-0 #GoKingsGo
Period: 2nd-20:00, Period Official-Period Official #GoKingsGo
Los Angeles Kings-1, Arizona Coyotes-0
NOT TWEETED---Period: 3rd-00:00, Period Ready-Period Ready
Period Starting: 3rd --- Score: Los Angeles Kings-1, Arizona Coyotes-0 #GoKingsGo
Period: 3rd-00:00, Faceoff-Travis Boyd faceoff won against Phillip Danault #GoKingsGo
Period: 3rd-00:16, Penalty-Phil Kessel Minor against Alexander Edler #GoKingsGo
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angel

Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Period: 3rd-01:31, Shot-M

Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Arizona Coyotes-0
Los Angeles Kings-1, Ariz

Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Ariz

Period: 3rd-05:43, Blocked Shot-Tobias Bjornfot shot blocked shot by Antoine Roussel #GoKingsGo
Period: 3rd-05:59, Missed Shot-Mikey Anderson Missing key [ED_null] Karel Vejmelka #GoKingsGo
NOT TWEETED---Period: 3rd-06:01, Giveaway-Giveaway by Kyle Capobianco
Period: 3rd-06:03, Shot-Adrian Kempe Snap Shot saved by Karel Vejmelka #GoKingsGo
Period: 3rd-06:04, Stoppage-Goalie Stopped #GoKingsGo
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1, Arizona Coyotes-1
Los Angeles Kings-1

...

Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles

Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles

Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles

Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles

Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles Kings-0, Toronto Maple Leafs-0
Los Angeles

In [9]:
#use to catch up to live
for play in allPlays:
    if f'Period: {period}-{timeee}, {event}' not in gamedetail:
        gamedetail.append(f'Period: {period}-{timeee}, {event}')
        print(play)

In [13]:
#print(x)
#x=x+1
pullurl = starturl + str(gameid) + endurl
try:
    with urllib.request.urlopen(pullurl) as url:
        obj = json.loads(url.read().decode())
except:
    print(f'The URL is broke, bruh. Try again.')
allPlays = obj["liveData"]["plays"]['allPlays']
hometeamscore = 0
awayteamscore = 0
for play in allPlays:
    period = play['about']['ordinalNum']
    event = play['result']['event']
    timeee = play['about']['periodTime']
    description = play['result']['description']
    if event == 'Goal':
        if str(home_team) == 'Los Angeles Kings':
            if play['team']['name'] == 'Los Angeles Kings':
                hometeamscore += 1
            else:
                awayteamscore += 1
        if str(home_team) != 'Los Angeles Kings':
            if play['team']['name'] != 'Los Angeles Kings':
                hometeamscore += 1
            else:
                awayteamscore += 1
    if f'Period: {period}-{timeee}, {event}' not in gamedetail:
        gamedetail.append(f'Period: {period}-{timeee}, {event}')
        if event == 'Period Ready' or event == 'Hit' or event == 'Giveaway' or event == 'Takeaway' or event == 'Game Scheduled':
            print(f'NOT TWEETED---Period: {period}-{timeee}, {event}-{description}')
            #Period Ready
            #Giveaway
            #Takeaway
            #Hit
        #elif event == 'Game Scheduled':
            #homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
            #awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
            #print(f'Scheduled Game: {home_team} at {away_team} #GoKingsGo')
            #api.update_status(f'Scheduled Game: {home_team} versus {away_team} #GoKingsGo')
            #time.sleep(8)
        elif event == 'Period Start':
            try:
                #api.update_status(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                print(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
            except:
                print(f'FAILED TO TWEET --- Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
            time.sleep(8)
            #Period Start
        elif event == 'Faceoff' or event == 'Blocked Shot' or event == 'Stoppage' or event == 'Period Official' or event == 'Game End' or event == 'Game Official' or event == 'Shot' or event == 'Penalty' or event == 'Missed Shot' :
            try:
                #api.update_status(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                print(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
            except:
                print(f'FAILED TO TWEET -- Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
            #time.sleep(8)
            #Faceoff
            #Blocked Shot
            #Shot
            #Penalty
            #Missed Shot
            #Stoppage
            #Period Official
            #Game End
            #Game Official
        elif event == 'Goal':
            try:
                #api.update_status(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                print(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
            except:
                print(f'FAILED TO TWEET --- Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
            #time.sleep(8)
            #Goal
        elif event == 'Period End' and (period == '1st' or period == '2nd'):
            try:
                #api.update_status(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                print(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
            except:
                print(f'FAILED TO TWEET --- Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
            #time.sleep(1080)
            #Period End
        elif event == 'Period End' and (period == '3rd'):
            try:
                #api.update_status(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                print(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
            except:
                print(f'FAILED TO TWEET --- End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')

            #time.sleep(8)
        elif event == 'Period End' and (period == 'OT'):
            try:
                #api.update_status(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                print(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
            except:
                print(f'FAILED TO TWEET --- End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
            #time.sleep(8)
        else:
            time.sleep(0)
print(f'{home_team}-{hometeamscore}, {away_team}-{awayteamscore}')

Period: 1st-15:52, Shot-Andreas Athanasiou Wrist Shot saved by Karel Vejmelka #GoKingsGo
Period: 1st-15:57, Shot-Brendan Lemieux Snap Shot saved by Karel Vejmelka #GoKingsGo
NOT TWEETED---Period: 1st-15:58, Hit-Brendan Lemieux hit Cam Dineen
NOT TWEETED---Period: 1st-17:17, Giveaway-Giveaway by Trevor Moore
NOT TWEETED---Period: 1st-17:19, Giveaway-Giveaway by Rasmus Kupari
NOT TWEETED---Period: 1st-17:46, Hit-Lawson Crouse hit Mikey Anderson
Period: 1st-17:56, Shot-Ryan Dzingel Snap Shot saved by Jonathan Quick #GoKingsGo
Period: 1st-18:08, Blocked Shot-Tobias Bjornfot shot blocked shot by Lawson Crouse #GoKingsGo
Period: 1st-18:15, Shot-Tobias Bjornfot Tip-In saved by Karel Vejmelka #GoKingsGo
Period: 1st-18:24, Blocked Shot-Viktor Arvidsson shot blocked shot by Shayne Gostisbehere #GoKingsGo
NOT TWEETED---Period: 1st-19:09, Hit-Alexander Edler hit Liam O'Brien
Period: 1st-19:11, Shot-Alexander Edler Wrist Shot saved by Karel Vejmelka #GoKingsGo
Period: 1st-19:12, Stoppage-Puck in Ne